In [112]:
# f = (f' - B - d * t) / med (f' - b - d * t)
# D = D' - B
# B = B' + Berr
# a few things to talk about 
# Getting into modeling
# Just set up the interns 

In [113]:
import numpy as np
import matplotlib.pyplot as plt

In [114]:
# parameters
# gain electrons per adu
g = 3.6
# dimension 
row = 2048
column = 2048
# bias levels 
# real bias level count per pixel 
bias_real_level = 300
# the systematic error on the bias
readnoise = 5
# assumed bias level float up to factor of 2 
factor = 2 
bias_meas_array = np.linspace(bias_real_level/factor, bias_real_level*factor)
# dark
# dark current elec per sec
d = 0.05
# time dark exposure
td = 60
# flat counts per pixel 
f = 10000
# time flat exposure 
tf_array = np.linspace(3, 120)

In [115]:
# function making a bias based on a given bias input
# normal distributed 
# output: 2d bias level in counts per pixel 
def makeBias(bias):
    bias = np.random.normal(bias, readnoise, (row, column))
    return bias

In [116]:
# dark function based on a given bias input
# possion distributed
# output: 2d dark current in counts per sec per pixel 
def makeDark(bias):
    dark = makeBias(bias) / td + np.random.poisson(d, (row, column)) / g
    return dark     

In [117]:
# flat function and possion given a bias, total counts 
# possion distributed 
# output: 2d flat level in counts per pixel 
def makeFlat(bias, tf):
    flat = makeBias(bias) + makeDark(bias) * tf  + np.random.poisson(f * g, (row, column)) / g
    return flat

In [118]:
# prop storage array
percent_change_in_bias_array = []
percent_change_in_flat_array = []
tf_plot_array = []

In [119]:
# making the real bias, dark and flat
bias_real = makeBias(bias_real_level)
dark_real = makeDark(bias_real_level)
flat_raw = makeFlat(bias_real_level)

In [120]:
# prop 
for b in bias_meas_array:
    # measured bias 2d array
    bias_meas = makeBias(b)
    # bias displacement
    deltab = bias_real - bias_meas
    # bias percent change mean
    percent_change_in_bias = np.mean(deltab / bias_real * 100)
    # measured dark 
    dark_meas = makeDark(b)
    # measured flat
    flat_measured_raw = makeFlat(b)
    for tf in tf_array:
        # real flat in a 2d array form 
        flat_real = (flat_raw - dark_real * tf - bias_real) / np.median(flat_raw - dark_real * tf - bias_real)
        # flat measured in a 2d array form
        flat_meas = (flat_raw - dark_meas * tf - bias_meas) / np.median(flat_raw - dark_meas * tf - bias_meas)
        # flat displacement in a 2d 
        deltaf = flat_real - flat_meas
        # flat percent change in a mean form 
        percent_change_in_flat = np.mean(deltaf / flat_meas * 100)
        percent_change_in_bias_array.append(percent_change_in_bias)
        percent_change_in_flat_array.append(percent_change_in_flat)
        tf_plot_array.append(tf)

KeyboardInterrupt: 

In [ ]:
ax = plt.axes(projection='3d')
# Adding data to our axes, 's=40' is to increase point size by 40
ax.scatter3D(percent_change_in_bias_array, tf_plot_array, percent_change_in_flat_array, s=1) 

# Labelling your axes
ax.set_xlabel('Percent change in Bias')
ax.set_ylabel('Time Flat Exposure')
ax.set_zlabel('Percent change in Flat')

In [ ]:
def func(xy, a, b, c, d, e, f):
    x, y = xy
    return a + b*x + c*y + d*x**2 + e*y**2 + f*x*y

In [ ]:
from scipy.optimize import curve_fit
  
# Perform curve fitting
popt, pcov = curve_fit(func, (percent_change_in_bias_array, tf_plot_array), percent_change_in_flat_array)
  
# Print optimized parameters
print(popt)

In [ ]:
X, Y = np.meshgrid(percent_change_in_bias_array, tf_plot_array)
pred_flat_change = func((X, Y), *popt)

In [ ]:
ax = plt.axes(projection='3d')
# Adding data to our axes, 's=40' is to increase point size by 40
ax.scatter3D(percent_change_in_bias_array, tf_plot_array, percent_change_in_flat_array, s=1) 
ax.plot_surface(X, Y, pred_flat_change, color='red', alpha=0.5)
# Labelling your axes
ax.set_xlabel('Percent change in Bias')
ax.set_ylabel('Time Flat Exposure')
ax.set_zlabel('Percent change in Flat')
ax.view_init(0, 45)

In [ ]:
func((50, 60), *popt)

In [33]:
# # prop
# for b in bias_meas_mean:
#     # bias measured in a 2d form with systematic error
#     bias_meas = np.array([[np.random.normal(b, bias_err) for _ in range(column)] for _ in range(row)])
#     # bias displacement in a 2d form 
#     deltab = bias_real - bias_meas
#     # bias change in a mean form
#     percent_change_in_bias = np.mean(deltab / bias_real * 100)
#     # measured dark 
#     d_meas = d / g - bias_meas / td
#     # for each flat exposure time 
#     for tf in tf_array:
#         # real flat in a 2d array form 
#         flat_real = (flat_raw - d_real * tf - bias_real) / np.median(flat_raw - d_real * tf - bias_real)
#         # flat measured in a 2d array form
#         flat_meas = (flat_raw - d_meas * tf - bias_meas) / np.median(flat_raw - d_meas * tf - bias_meas)
#         # flat displacement in a 2d 
#         deltaf = flat_real - flat_meas
#         # flat percent change in a mean form 
#         percent_change_in_flat = np.mean(deltaf / flat_meas * 100)
#         percent_change_in_bias_array.append(percent_change_in_bias)
#         percent_change_in_flat_array.append(percent_change_in_flat)
#         tf_plot_array.append(tf)

In [35]:
# # number of pixels in rows and columns, can change with the dimension of the CCD
# rows = 100
# columns = 100
# # dark counts per second per pixel 
# dark = 2 
# # the flactuation of flat with a 3% error
# flatUpper = 1.03
# flatLower = 0.97
# # bias counts and overall bias level change
# bias = 4000
# biasChangeUpper = 0.2
# biasChangeLower = 0.01
# # integration time 
# time = 300 

# # flat field randomization
# flatField = np.matrix([[np.random.uniform(flatLower, flatUpper) for _ in range(columns)] for _ in range(rows)])

In [36]:
# #change for the bias
# biasChange = np.linspace(biasChangeLower, biasChangeUpper)
# # flat field propagation 
# mean_displacement = []
# for b in biasChange:  
#     master_bias = np.matrix([[np.random.normal(bias, bias * b) for _ in range(columns)] for _ in range(rows)])
#     master_dark = dark * time + master_bias
#     flat = flatField - master_dark - master_bias
#     flat /= np.mean(flat)
#     displacement = np.mean(np.abs((flat - flatField)) / flatField)
#     mean_displacement.append(displacement) 
    
# plt.plot(biasChange, mean_displacement, 'o')
# plt.title("BiasPropagation")
# plt.xlabel('BiasChange')
# plt.ylabel('Displacement')
# plt.show()

In [37]:
# fit = np.polyfit(biasChange, mean_displacement,1,full=False, cov=True)

# # Fit parameters are the first element in the returned "tuple"
# fitparams = fit[0]
# slope = fitparams[0]
# intercept = fitparams[1]

# # Covariance matrix is the second element in the returned "tuple"
# cov = fit[1]

# # This is the way you get errors out of the covariance matrix.
# param_error = np.sqrt(np.diagonal(cov))
# slope_error = param_error[0]
# intercept_error = param_error[1]

# slope_output = 'slope is %.3f +/- %0.3f' %(slope,slope_error)
# intercept_output = 'intercept is %.3f +/- %0.3f' %(intercept,intercept_error)

# plt.errorbar(biasChange,mean_displacement)
# plt.xlabel('Bias')
# plt.ylabel('Mean Displacement (flat - flatPrime) / flatPrime')
# xfit = np.linspace(plt.xlim()[0],plt.xlim()[1],100)
# yfit = intercept + slope*xfit
# plt.plot(xfit,yfit,'r--', label='Linear output with ' + slope_output + intercept_output)
# plt.legend()

# print('The slope is %.3f +/- %0.3f' %(slope,slope_error))
# print('The intercept is %.3f +/- %0.3f' %(intercept,intercept_error))
# # plt.savefig('Updated Linear model')